- [previous file - EDA](2022-03-31_train-test_EDA.ipynb)
- [next tile - models with pre-tuned parameters and ensembles](2022-04-15_ensemble.ipynb)

## imports

In [7]:
from warnings import filterwarnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_percentage_error

from lib.model_related import *

sns.set()
filterwarnings("ignore")

## reading data

In [8]:
train_raw = pd.read_parquet("data/2022-04-08_train_pre-model.parquet")
test_raw = pd.read_parquet("data/2022-04-08_test_pre-model.parquet")

train_raw.shape, test_raw.shape

((115367, 30), (34686, 28))

In [9]:
def submit(hold_out: pd.DataFrame, model, name="submission"):
    preds = model.predict(hold_out)
    submission = pd.read_csv("data/sample_submission.csv")
    submission["price"] = preds
    submission.to_csv(f"{name}.csv", index=False)
    
    
def submit_log(hold_out: pd.DataFrame, model, name="submission"):
    preds = model.predict(hold_out)
    submission = pd.read_csv("data/sample_submission.csv")
    submission["price"] = np.exp(preds)
    submission.to_csv(f"{name}.csv", index=False)

## encoding

In [10]:
train_raw["train/test"] = "train"
test_raw["train/test"] = "test"

data = train_raw.append(test_raw)
data["ptc"].fillna("Оригинал", inplace=True)

data[data.select_dtypes("object").columns.tolist()] = data[
    data.select_dtypes("object").columns.tolist()
].astype(str)

for col in set(data.select_dtypes(exclude=("object")).columns) - {"price"}:
    data[col] = (
        RobustScaler().fit_transform(data[col].values.reshape(-1, 1)).reshape(-1, 1)
    )

for col in ["model_name"]:
    data[col] = LabelEncoder().fit_transform(data[col].astype("str"))

data = pd.get_dummies(
    data,
    columns=[
        "vehicle_transmission",
        "vendor",
        "brand",
        "fuel_type",
        "body_type",
        "color",
        "ptc",
        "drive",
        "wheel",
        "age_cat",
    ],
)

train = data.loc[data["train/test"] == "train"]

train_jane = train.loc[train["sample"] == "jane"]
train_sokolov = train.loc[train["sample"] == "sokolov"]
train_jane["price"] = train_jane["price"] * 0.86
train = train_jane.append(train_sokolov)

train.drop(columns=["sample", "description", "train/test"], inplace=True)
test = data.loc[data["train/test"] == "test"].drop(
    columns=["sample", "description", "train/test", "price"]
)

## modelling

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns="price"), train["price"], random_state = 42, shuffle=True)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((86525, 112), (86525,), (28842, 112), (28842,))

## model tuning

### Extra Tree Regressor

In [ ]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 14),
        'max_samples': trial.suggest_uniform('max_samples', 0.6, 0.99),
        'max_features': trial.suggest_categorical("max_features", ["auto", "sqrt", 'log2']),
        'max_depth': None,
        'bootstrap': True,
        'random_state': 42
    }

    etr_o = ExtraTreesRegressor(**param)
    cv_roc_auc = cross_val_score(etr_o, X_train, y_train, cv=3, scoring="neg_mean_absolute_percentage_error", n_jobs=-1)

    return np.mean(cv_roc_auc)


study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ETRRegressor.db",
    study_name="ETRRegressor",
    load_if_exists=True,
)
study.optimize(objective, n_trials=200)

[I 2022-04-14 11:02:52,619] Using an existing study with name 'ETRRegressor' instead of creating a new one.
[I 2022-04-14 11:20:48,809] Trial 44 finished with value: -0.14031671545863925 and parameters: {'n_estimators': 950, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_samples': 0.9666845348937461, 'max_features': 'auto'}. Best is trial 25 with value: -0.1397990866734321.
[I 2022-04-14 11:34:15,964] Trial 45 finished with value: -0.1427228867461674 and parameters: {'n_estimators': 829, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_samples': 0.9302304299278186, 'max_features': 'auto'}. Best is trial 25 with value: -0.1397990866734321.
[I 2022-04-14 11:35:58,464] Trial 46 finished with value: -0.1419002199476845 and parameters: {'n_estimators': 105, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_samples': 0.635460897607993, 'max_features': 'auto'}. Best is trial 25 with value: -0.1397990866734321.
[I 2022-04-14 11:37:49,319] Trial 47 finished with value: -0.259587979866

#### Version 1 (tuned by hand)

In [34]:
etr_cust = ExtraTreesRegressor(
    n_estimators=800,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='auto',
    max_depth=15,
    bootstrap=True,
    random_state=42, 
    n_jobs=-1,
    verbose=0
).fit(X_train, np.log(y_train))

print("etr_cust_log", mean_absolute_percentage_error(y_valid, np.exp(etr_cust.predict(X_valid))))
submit_log(test, etr_cust, "etr_cust_log")
# etr_cust_log 0.1376278076519815

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  5.1min finished


#### Version 2 (optuna)

In [13]:
etr_tuned_78 = ExtraTreesRegressor(
    **{
        'n_estimators': 936, 
        'min_samples_split': 3, 
        'min_samples_leaf': 1, 
        'max_samples': 0.9894458395539251, 
        'max_features': 'auto',
        'max_depth': None,
        'bootstrap': True,
        'random_state': 42,
        'n_jobs': -1,
        'verbose': 0
    }
).fit(X_train, np.log(y_train))

print("etr_tuned_78_log", mean_absolute_percentage_error(y_valid, np.exp(etr_tuned_78.predict(X_valid))))
submit_log(test, etr_tuned_78, "etr_tuned_78_log")

etr_tuned_78_log 0.1243004166689037
